In [2]:
import urllib.request
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

import time
import re
import argparse
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
client_id = 'hwEnzlSdGXZTyeubesmy'
client_secret = 'WkpCvqbw6u'

In [ ]:
### Naver API에서 뉴스 항목을 불러옵니다.

def get_news_list(crawling_trg, client_id, client_secret):
    news_list = []
    for step in range(1, 1100, 100):
        if step == 1001:
            step = 1000
        
        params = dict(
            query=urllib.parse.quote(crawling_trg),
            display=urllib.parse.quote('100'), 
            start=urllib.parse.quote(str(step)),
            sort=urllib.parse.quote('date'),
        )

        url = f"https://openapi.naver.com/v1/search/news?query={params['query']}&display={params['display']}&start={params['start']}&sort={params['sort']}"

        request = urllib.request.Request(url)
        request.add_header('X-Naver-Client-Id',client_id)
        request.add_header('X-Naver-Client-Secret',client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if(rescode==200):
            response_body = response.read()
            result = json.loads(response_body.decode('utf-8'))['items']
            news_list.extend(result)
        else:
            print('Error Code:' + rescode)
        # print(step)
        
        time.sleep(5)
        
        if step == 101:
            break
    
    news_list_df = pd.DataFrame(news_list).drop_duplicates()
    news_list_df['checker'] = news_list_df.link.str.find('https://n.news.naver.com/')
    news_list_df = news_list_df[news_list_df['checker'] > -1]
    news_list_df.reset_index(inplace=True)
    news_list_df['crawling_trg'] = crawling_trg
    
    return news_list_df

In [ ]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'cookie': 'nid_inf=1417715440; NID_AUT=jB6JKkl0bQJkzabA2sqcsZPcc58lN3Nk/oKjGyVYl1n8PedB7ujlxHdECKcm9kS7; NID_JKL=CdIfrElqYRkes7uT/S10prytLgVd7C1HBLS0nz55Sys=; VISIT_LOG_CLEAN=1; NFS=2; NID_SES=AAABkxQsSdlab5VkBVUMJHd12mikXaT/HRo/rtzzaxgjXmXJLfOFG25S5TiaZsFNk5bLeRF4Ej3UoZTevQC7Jd7fVFOAFJtSFoNgapi8qqUTVHDqzpWo4zUjmx/LD5cdsWbvaL/cByCai8XYLRLdB0o6W7sQutPmRMFUblZMKU/CdpBh8aziLA/i9wHRgkCr4zpYxc/YROLyZLZ7QmphI5qakKYHN4OxNKYy/Dp+PvlCsWk6azoXw7AxABwR/NdLMwDxNgZ6yN0iUJDkWFFPEN0yBGCm5VaOa1oF4tU0GkLMQsbyMb/+mOw6aBjxQNSgS5RDsn3Vs6H9l1qczNtasU4OpdonIK0EzoLYyQEgLkpXeLTnzkdaJnqR7EyjKjl8hH/EsaVdnRaAEs0afY3VK8f+pCn0z/5n9xpiCd0UVrpoI3nwF1No4x/mEk71EW2ghH90hyg/9F0KHAHLneYZ6lCvcdhE/ENHrmc1nP0R0sUnOltV0XIyf0fEua7rvSqv0P0iuE0xD2+WUqh//96WFUP+XyeQxvn16XPBT8PSfE/nU/FQ; nx_ssl=2; page_uid=h1WRNsp0YiRssKvZbrNssssssMh-389234; BMR='
}

In [ ]:
### Naver API에서 불러온 뉴스 항목의 본문을 불러옵니다.

def crawl_news(news_list_df, headers):
    crawled_news = []
    for idx, row in news_list_df.iterrows():
        # row.crawling_trg, row.pubDate, row.title, cleansed_news_body, row.originallink, row.link, row.description
        URL = str(row.link)
        r = requests.get(URL, headers=headers)
        
        try:
            soup = BeautifulSoup(r.content, 'html5lib')
            news_body = soup.select('#newsct_article')[0].text
            cleansed_news_body = re.sub(r'[\n\t^/$]', '', news_body)
            crawled_news.append([row.crawling_trg, row.pubDate, row.title, cleansed_news_body, row.originallink, row.link, row.description])
        except Exception as e:
            soup = BeautifulSoup(r.content, 'html5lib')
            news_body = soup.select('#articeBody')[0].text
            cleansed_news_body = re.sub(r'[\n\t^/$]', '', news_body)
            crawled_news.append([row.crawling_trg, row.pubDate, row.title, cleansed_news_body, row.originallink, row.link, row.description])
        
        time.sleep(5)
        
        if idx == 3:
            break
    
    crawled_news_df = pd.DataFrame(crawled_news, columns=['crawling_trg', 'pubDate', 'title', 'body', 'originallink', 'link', 'description'])
    return crawled_news_df

In [9]:
with open('government_organization_chart.json', encoding='utf-8') as f:
    gov_orgs = json.load(f)

In [10]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--query", dest="query", action="store")
# args = parser.parse_args()

query = '인공지능'
org_name = '기획재정부'
sub_orgs = gov_orgs[org_name]

crawling_trgs = []
for sub_org in sub_orgs:
    crawling_trg = ' '.join([org_name, sub_org, query])
    crawling_trgs.append(crawling_trg)

In [ ]:
crawled_news_dfs = []
for crawling_trg in crawling_trgs:
    news_list_df = get_news_list(crawling_trg, client_id, client_secret)
    crawled_news_df = crawl_news(news_list_df, headers)
    crawled_news_dfs.append(crawled_news_df)
    
result = pd.concat(crawled_news_dfs)

In [46]:
search_sub_org = result.body.str.contains('|'.join(sub_orgs), case=False, regex=True)
search_query = result.body.str.contains(query, case=False, regex=True)
result = result[search_sub_org & search_query]

In [47]:
result.to_csv(f'{"_".join([query, org_name])}_crawled.csv', index=False)

### Test

In [45]:
result =pd.read_csv('인공지능_기획재정부_crawled.csv')

In [40]:
query, sub_orgs

('인공지능',
 ['한국수출입은행',
  '한국조폐공사',
  '한국재정정보원',
  '한국투자공사',
  '복권위원회사무처',
  '국세청',
  '관세청',
  '조달청',
  '통계청'])

In [41]:
len(result)

15

In [42]:
search_sub_org = result.body.str.contains('|'.join(sub_orgs), case=False, regex=True)
search_query = result.body.str.contains(query, case=False, regex=True)
len(result[search_sub_org & search_query])

11

In [43]:
search_condition = (result.body.str.contains('|'.join(sub_orgs), case=False, regex=True)) & (result.body.str.contains(query, case=False, regex=True))
len(result[search_condition])

11

In [44]:
len(result)

15